# Installing Whisper

The commands below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [1]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

  Cloning https://github.com/openai/whisper.git to c:\users\sumesh\appdata\local\temp\pip-req-build-u3jy807y
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
      --------------------------------------- 0.1/5.7 MB 4.3 MB/s eta 0:00:02
     - -

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\Sumesh\AppData\Local\Temp\pip-req-build-u3jy807y'

[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   ------------------------------------- -- 92.2/97.9 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 97.9/97.9 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.6 MB 2.6 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/1.6 MB 2.5 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.6 MB 2.4 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.6 MB 3.5 MB/s eta 0:00:01
   --------------------- ------------------ 0.9/1.6 MB 4.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.6 MB 4.2 MB/s eta 0:00:01
   ------------------------------- -------- 1.3/1.6 MB 4.5 MB/s eta 0:00:01
   -------------------------------------- - 1.6/1.6 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Loading the LibriSpeech dataset

The following will load the test-clean split of the LibriSpeech corpus using torchaudio.

In [3]:
!pip install torchaudio

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.4 MB 1.1 MB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.4 MB 2.0 MB/s eta 0:00:02
   ------- -------------------------------- 0.4/2.4 MB 3.4 MB/s eta 0:00:01
   ---------- ----------------------------- 0.6/2.4 MB 3.8 MB/s eta 0:00:01
   ------------- -------------------------- 0.8/2.4 MB 3.9 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.4 MB 4.1 MB/s eta 0:00:01
   --------------------- ------------------ 1.2/2.4 MB 4.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.5/2.4 MB 4.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.4 MB 4.6 MB/s eta 0:00:01
   ------------------------------- -------- 1.8/2.4 MB 4.5 MB/s eta 0:00:01
   ---------------------------------- ----- 2.0/2.4 MB 4.6 MB/s eta 0:00:01
   -----------------------


[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
class LibriSpeech(torch.utils.data.Dataset):
    """
    A simple class to wrap LibriSpeech and trim/pad the audio to 30 seconds.
    It will drop the last few seconds of a very small portion of the utterances.
    """
    def __init__(self, split="test-clean", device=DEVICE):
        self.dataset = torchaudio.datasets.LIBRISPEECH(
            root=os.path.expanduser("~/.cache"),
            url=split,
            download=True,
        )
        self.device = device

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        audio, sample_rate, text, _, _, _ = self.dataset[item]
        assert sample_rate == 16000
        audio = whisper.pad_or_trim(audio.flatten()).to(self.device)
        mel = whisper.log_mel_spectrogram(audio)
        
        return (mel, text)

In [6]:
dataset = LibriSpeech("test-clean")
loader = torch.utils.data.DataLoader(dataset, batch_size=16)

100%|██████████| 331M/331M [02:56<00:00, 1.97MB/s] 


# Running inference on the dataset using a base Whisper model

The following will take a few minutes to transcribe all utterances in the dataset.

In [7]:
model = whisper.load_model("base.en")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|███████████████████████████████████████| 139M/139M [00:29<00:00, 4.88MiB/s]


Model is English-only and has 71,825,408 parameters.


In [8]:
# predict without timestamps for short-form transcription
options = whisper.DecodingOptions(language="en", without_timestamps=True)

In [10]:
!pip install librosa

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.4 kB 890.4 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 809.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     --- ------------------------------------ 0.1/1.0 MB 5.1 MB/s eta 0:00:01
     ------------ --------------------------- 0.3/1.0 MB 4.7 MB/s eta 0:00:01
     ---------------------- ----------------- 0.6/1.0 MB 6.0 MB/s eta 0:00:01
     ------------------------------ --------- 0.8/1.0 MB 5.4 MB/s eta 0:00:01
     ------------------------------------- -- 1.0/1.0 MB 5.5 MB/s eta 0:00:01
     ---------------------------------------- 1.0/1.0 MB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/253.7 kB ? eta -:--:--
   ----------------------------------- ---- 225.3/253.7 kB 6.9 MB/s eta 0:00:01
   --


[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install soundfile


[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!pip install ffmpeg

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6089 sha256=56dd43f5a202a68fda5ab3734544e25dc23128d365cdcf3957a69bcac5240dcd
  Stored in directory: c:\users\sumesh\appdata\local\pip\cache\wheels\26\21\0c\c26e09dff860a9071683e279445262346e008a9a1d2142c4ad
Successfully built ffmpeg



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
!pip install SoundFile



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install ffmpeg-python


     ---------------------------------------- 0.0/840.9 kB ? eta -:--:--
     ----- -------------------------------- 112.6/840.9 kB 3.3 MB/s eta 0:00:01
     ------------- ------------------------ 307.2/840.9 kB 3.8 MB/s eta 0:00:01
     ----------------------- -------------- 522.2/840.9 kB 4.7 MB/s eta 0:00:01
     ------------------------------- ------ 696.3/840.9 kB 4.9 MB/s eta 0:00:01
     -------------------------------------- 840.9/840.9 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492054 sha256=7a184cf2ee957bd67e60229cfde0de43af4d77710dfb7c64a221327606913245
  Stored in directory: c:\users\sumesh\appdat


[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
hypotheses = []
references = []

for mels, texts in tqdm(loader):
    results = model.decode(mels, options)
    hypotheses.extend([result.text for result in results])
    references.extend(texts)

  0%|          | 0/164 [00:00<?, ?it/s]

RuntimeError: Couldn't find appropriate backend to handle uri C:\Users\Sumesh/.cache\LibriSpeech\test-clean\1089\134686\1089-134686-0000.flac and format None.

In [8]:
data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references))
data

,hypothesis,reference
0,"He hoped there would be stew for dinner, turni...",HE HOPED THERE WOULD BE STEW FOR DINNER TURNIP...
1,"Stuffered into you, his belly counseled him.",STUFF IT INTO YOU HIS BELLY COUNSELLED HIM
2,After early nightfall the yellow lamps would l...,AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD L...
3,"Hello Bertie, any good in your mind?",HELLO BERTIE ANY GOOD IN YOUR MIND
4,Number 10. Fresh Nelly is waiting on you. Good...,NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD ...
...,...,...
2615,"Oh, to shoot my soul's full meaning into futur...",OH TO SHOOT MY SOUL'S FULL MEANING INTO FUTURE...
2616,"Then I, long tried by natural ills, received t...",THEN I LONG TRIED BY NATURAL ILLS RECEIVED THE...
2617,I love thee freely as men strive for right. I ...,I LOVE THEE FREELY AS MEN STRIVE FOR RIGHT I L...
2618,"I love thee with the passion put to use, in my...",I LOVE THEE WITH THE PASSION PUT TO USE IN MY ...


# Calculating the word error rate

Now, we use our English normalizer implementation to standardize the transcription and calculate the WER.

In [9]:
import jiwer
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

In [10]:
data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
data["reference_clean"] = [normalizer(text) for text in data["reference"]]
data

,hypothesis,reference,hypothesis_clean,reference_clean
0,"He hoped there would be stew for dinner, turni...",HE HOPED THERE WOULD BE STEW FOR DINNER TURNIP...,he hoped there would be stew for dinner turnip...,he hoped there would be stew for dinner turnip...
1,"Stuffered into you, his belly counseled him.",STUFF IT INTO YOU HIS BELLY COUNSELLED HIM,stuffered into you his belly counseled him,stuff it into you his belly counseled him
2,After early nightfall the yellow lamps would l...,AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD L...,after early nightfall the yellow lamps would l...,after early nightfall the yellow lamps would l...
3,"Hello Bertie, any good in your mind?",HELLO BERTIE ANY GOOD IN YOUR MIND,hello bertie any good in your mind,hello bertie any good in your mind
4,Number 10. Fresh Nelly is waiting on you. Good...,NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD ...,number 10 fresh nelly is waiting on you good n...,number 10 fresh nelly is waiting on you good n...
...,...,...,...,...
2615,"Oh, to shoot my soul's full meaning into futur...",OH TO SHOOT MY SOUL'S FULL MEANING INTO FUTURE...,0 to shoot my soul is full meaning into future...,0 to shoot my soul is full meaning into future...
2616,"Then I, long tried by natural ills, received t...",THEN I LONG TRIED BY NATURAL ILLS RECEIVED THE...,then i long tried by natural ills received the...,then i long tried by natural ills received the...
2617,I love thee freely as men strive for right. I ...,I LOVE THEE FREELY AS MEN STRIVE FOR RIGHT I L...,i love thee freely as men strive for right i l...,i love thee freely as men strive for right i l...
2618,"I love thee with the passion put to use, in my...",I LOVE THEE WITH THE PASSION PUT TO USE IN MY ...,i love thee with the passion put to use in my ...,i love thee with the passion put to use in my ...


In [11]:
wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

print(f"WER: {wer * 100:.2f} %")

WER: 4.26 %
